In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class SolarRoofLoading(object):
    
    def __init__(self, cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length):
        # Initialises the loading values for the roof type
        self.cladding = cladding
        self.services = services
        self.PV_loading = PV_loading
        self.maintenance = maintenance
        self.uniform_snow_loading = uniform_snow_loading
        self.snow_drift_valley = snow_drift_valley
        self.snow_drift_parapet = snow_drift_parapet
        self.wind_down = wind_down
        self.snow_drift_length = snow_drift_length
        
        # Initialises different combinations of loads
        self.dead_loading = self.cladding + self.services
        self.existing_loading =  self.dead_loading + self.maintenance
        self.proposed_loading = self.dead_loading + self.PV_loading
        
        # Initialises empty pandas series and dictionaries to store values of roof survey
        self.distance_to_next_purlin = pd.Series(dtype=float)
        self.distance_from_ridge = pd.Series(dtype=float)
        self.load_width = pd.Series(dtype=float)
        self.purlin_ULS_capacity = pd.Series(dtype=float)
        self.purlin_SLS_capacity = pd.Series(dtype=float)
        self.purlin_capacities = {}
        self.keys = ['ULS Capacity', 'SLS Capacity']
        
        
        # Ask user to input the roof structures measurements from the survey 
        self.purlin_no = int(input('Please enter the number of purlins: '))
        self.span = float(input('Please enter the max purlin span: '))
        self.purlin_type = str(input('Please enter purlin type: '))
        
        
        # Ask user to input the spacing between each purlin starting from the lower edge
        for i in range(self.purlin_no):
            value = float(input(f'Input the spacing between purlin {i} to purlin {i+1}: '))
            self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i])) 
        
        
        # Ask user to input the ULS and SLS capacities
        for key in self.keys:
            self.purlin_capacities[key] = int(input(f'Enter {key} of the purlins: '))
        
        
        # Creates a series of the ULS capacities the same length as the number of purlins to compare 
        # against calculated values in the final table
        for i in range(1, self.purlin_no):
            value = self.purlin_capacities['ULS Capacity']
            self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
        self.purlin_ULS_capacity.name = (self.purlin_type + ' ULS Capacity (kN)')
        
        
        # Creates a series of the SLS capacities the same length as the number of purlins to compare 
        # against calculated values in the final table
        for i in range(1, self.purlin_no):
            value = self.purlin_capacities['SLS Capacity']
            self.purlin_SLS_capacity = self.purlin_SLS_capacity.append(pd.Series([value], index=[i]))
        self.purlin_SLS_capacity.name = (self.purlin_type + ' SLS Capacity (kN)')
        
        
        # Calculates the distance each purlin is from the ridge
        self.distance_from_ridge = self.distance_from_ridge.append(pd.Series([self.distance_to_next_purlin.sum()], index=[0]))
        for i in range(1, self.purlin_no):
            value = self.distance_from_ridge[i-1] - self.distance_to_next_purlin[i-1]
            self.distance_from_ridge = self.distance_from_ridge.append(pd.Series([value], index=[i]))
        
        
        # Calculates the load width for each purlin based on their spacing between adjacent purlins
        self.load_width = self.load_width.append(pd.Series([self.distance_to_next_purlin[0] + self.distance_to_next_purlin[1]/2], index=[1]))
        for i in range(2, self.purlin_no):
            value = (self.distance_to_next_purlin[i-1] + self.distance_to_next_purlin[i]) /2
            self.load_width = self.load_width.append(pd.Series([value], index=[i]))
        
    def load_increase_typical_area(self):
        sum_1 = self.existing_loading
        sum_2 = self.proposed_loading + self.uniform_snow_loading
        
        percentage_increase_typical = ((sum_2 - sum_1) / sum_1) * 100
        return percentage_increase_typical
        
        
    def load_increase_snow_valley(self):
        sum_1 = self.dead_loading + self.snow_drift_valley
        sum_2 = self.proposed_loading + self.snow_drift_valley

        percentage_increase_valley = ((sum_2 - sum_1) / sum_1) * 100
        return percentage_increase_valley
        
        
    def load_increase_snow_parapet(self):
        sum_1 = self.dead_loading + self.snow_drift_parapet
        sum_2 = self.proposed_loading + self.snow_drift_parapet

        percentage_increase_parapet = ((sum_2 - sum_1) / sum_1) * 100
        return percentage_increase_parapet
        
        
    def existing_LC_BS(self,select):
        # Creates an empty dictionary to store the exisitng conditions load combination with 
        # Britist Standard factors of safety (exist_lc_bs)
        exist_lc_bs = {}

        # Creating new key value pairs for dictionary 'existing_lc' and calculating load combinations values
        exist_lc_bs['exist_lc_1'] = 1.4 * self.dead_loading + 1.6 * (max(self.maintenance, self.uniform_snow_loading))
        exist_lc_bs['exist_lc_2'] = 1.4 * self.dead_loading + 1.4 * self.wind_down
        exist_lc_bs['exist_lc_3'] = 1.2 * self.dead_loading + 1.2 * self.wind_down + 1.2 * (max(self.maintenance, self.uniform_snow_loading))
        exist_lc_bs['exist_lc_4'] = 1.4 * self.dead_loading + 1.05 * self.snow_drift_valley
        exist_lc_bs['exist_lc_5'] = 1.4 * self.dead_loading + 1.05 * self.snow_drift_parapet
        exist_lc_bs['exist_lc_6'] = 1 * self.dead_loading + 1 * (max(self.maintenance, self.uniform_snow_loading))
        exist_lc_bs['exist_lc_7'] = 1 * self.dead_loading + 1 * self.uniform_snow_loading
        exist_lc_bs['exist_lc_8'] = 1 * self.dead_loading + 0.8 * self.wind_down + 0.8 * (max(self.maintenance, self.uniform_snow_loading))

        # Finding max values for certain load combinations
        exist_lc_bs['exist_max_1'] = (max(exist_lc_bs['exist_lc_1'],exist_lc_bs['exist_lc_2'],exist_lc_bs['exist_lc_3'])) 
        exist_lc_bs['exist_max_2'] = exist_lc_bs['exist_lc_4']
        exist_lc_bs['exist_max_3'] = exist_lc_bs['exist_lc_5']
        exist_lc_bs['exist_max_4'] = ((max(exist_lc_bs['exist_lc_6'],exist_lc_bs['exist_lc_7'],exist_lc_bs['exist_lc_8'])))
        
        return exist_lc_bs[select]
    
    
    def proposed_LC_BS(self, select):
        # Creates an empty dictionary to store the proposed conditions load combination with 
        # Britist Standard factors of safety (prop_lc_bs)
        prop_lc_bs = {}

        # Creating new key value pairs for dictionary 'existing_lc' and calculating load combinations values
        prop_lc_bs['prop_lc_1'] = 1.4 * self.dead_loading + 1.6 * (max(self.maintenance,self.uniform_snow_loading))
        prop_lc_bs['prop_lc_2'] = 1.4 * self.dead_loading + 1.4 * self.wind_down
        prop_lc_bs['prop_lc_3'] = 1.2 * self.dead_loading + 1.2 * self.wind_down + 1.2 * (max(self.maintenance,self.uniform_snow_loading))
        prop_lc_bs['prop_lc_4'] = 1.4 * self.dead_loading + 1.05 * self.snow_drift_valley
        prop_lc_bs['prop_lc_5'] = 1.4 * self.dead_loading + 1.05 * self.snow_drift_parapet
        prop_lc_bs['prop_lc_6'] = 1 * self.dead_loading + 1 * (max(self.maintenance,self.uniform_snow_loading))
        prop_lc_bs['prop_lc_7'] = 1 * self.dead_loading + 1 * self.uniform_snow_loading
        prop_lc_bs['prop_lc_8'] = 1 * self.dead_loading + 0.8 * self.wind_down + 0.8 * (max(self.maintenance,self.uniform_snow_loading))

        # Finding max values for certain groups of load combinations
        prop_lc_bs['prop_max_1'] = (max(prop_lc_bs['prop_lc_1'],prop_lc_bs['prop_lc_2'],prop_lc_bs['prop_lc_3'])) 
        prop_lc_bs['prop_max_2'] = prop_lc_bs['prop_lc_4']
        prop_lc_bs['prop_max_3'] = prop_lc_bs['prop_lc_5']
        prop_lc_bs['prop_max_4'] = (max(prop_lc_bs['prop_lc_6'],prop_lc_bs['prop_lc_7'],prop_lc_bs['prop_lc_8'])) 

        return prop_lc_bs[select]

In [3]:
class PurlinLoadValley(SolarRoofLoading):

    def __init__(self,cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length):        
        self.snow_drift_valley_loads = pd.Series(dtype=float)
        self.ULS_load_valley = pd.Series(dtype=float)    
        self.total_ULS_load_valley = pd.Series(dtype=float)
        self.ULS_check = pd.Series(dtype=float)
        super().__init__(cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length)
    
    
    def purlin_load_valley_calcs(self):   
        # Calculates snow drift load based on distance from the ridge
        for i in range(1, self.purlin_no):
            value = (self.snow_drift_valley * self.distance_from_ridge[i]) / self.distance_from_ridge[0]
            self.snow_drift_valley_loads = self.snow_drift_valley_loads.append(pd.Series([value], index=[i]))         
        self.snow_drift_valley_loads.name = 'Snow drift loading on purlin (kPa)'

        # Calculates ULS load for each purlin from the snow drift for that purlin and the dead loading
        for i in range(1, self.purlin_no):
            value = 1.4 * self.dead_loading + 1.05 * self.snow_drift_valley_loads[i] 
            self.ULS_load_valley = self.ULS_load_valley.append(pd.Series([value], index=[i]))
        self.ULS_load_valley.name = 'ULS area load on valley purlins (kPa)'

        # Calculates the total ULS load on each purlin
        for i in range(1, self.purlin_no):
            value = self.span * self.ULS_load_valley[i] * self.load_width[i]
            self.total_ULS_load_valley = self.total_ULS_load_valley.append(pd.Series([value], index=[i]))  
        self.total_ULS_load_valley.name = 'Total ULS load on valley purlins (kN)'

        # Checks the calcualted loading against the ULS capacity to be less than the design load
        for i in range(1, self.purlin_no):
            if self.total_ULS_load_valley[i] <= self.purlin_capacities['ULS Capacity']:
                value = 'OK'
            else:
                value = 'Not OK'
            self.ULS_check = self.ULS_check.append(pd.Series([value], index=[i]))
        self.ULS_check.name = 'Capacity'

        # Combines results of each series and brings them into 1 data frame
        df_purlin_load_valley = pd.concat([self.snow_drift_valley_loads, self.ULS_load_valley, self.total_ULS_load_valley, self.purlin_ULS_capacity, self.ULS_check], axis=1)

        return df_purlin_load_valley

In [4]:
class PurlinLoadParapet(SolarRoofLoading):
    
    def __init__(self,cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length):
        self.snow_drift_parapet_loads = pd.Series(dtype=float)
        self.ULS_load_parapet = pd.Series(dtype=float)   
        self.total_ULS_load_parapet = pd.Series(dtype=float)
        self.dist_from_end_drift = {}
        self.ULS_check = pd.Series(dtype=float)
        super().__init__(cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length)
    
    
    def purlin_load_parapet_calcs(self):
        # Works out the distance at which the parapet snow drift will reduced linearly to 0
        summ = 0    
        while summ <= self.snow_drift_length:  
            for i in range(self.purlin_no):
                summ += self.distance_to_next_purlin[i]
                drift_end = i

        # Works out distance of each purlin from the end of the snow drift
        self.dist_from_end_drift[1] = self.snow_drift_length - self.distance_to_next_purlin[0]
        for i in range(2, drift_end):
            self.dist_from_end_drift[i] = self.dist_from_end_drift[i-1] - self.distance_to_next_purlin[i-1]

        # Calculates the snow drift parapet loading on each purlin reducing linearly to 0 from the parapet
        for i in range(1, drift_end):
            value = self.snow_drift_parapet * self.dist_from_end_drift[i] / self.snow_drift_length
            self.snow_drift_parapet_loads = self.snow_drift_parapet_loads.append(pd.Series([value], index=[i]))      
        self.snow_drift_parapet_loads.name = 'Snow drift loading from parapet (kPa)'

        # Calculates ULS area load for each purlin from the snow drift for that purlin and the dead loading
        for i in range(1, drift_end):
            value = 1.4 * self.dead_loading + 1.05 * self.snow_drift_parapet_loads[i]
            self.ULS_load_parapet = self.ULS_load_parapet.append(pd.Series([value], index=[i])) 
        self.ULS_load_parapet.name = 'ULS area load on parapet purlins (kPa)'  

        # Calculates the total ULS load on each purlin from the parapet snow loading
        for i in range(1, drift_end):
            value = self.span * self.ULS_load_parapet[i] * self.load_width[i]
            self.total_ULS_load_parapet = self.total_ULS_load_parapet.append(pd.Series([value], index=[i]))
        self.total_ULS_load_parapet.name = 'Total ULS load on parapet purlins (kN)'

        # Checks the calcualted loading against the ULS capacity to be less than the design load
        for i in range(1, drift_end):
            if self.total_ULS_load_parapet[i] <= self.purlin_capacities['ULS Capacity']:
                value = 'OK'
            else:
                value = 'Not OK'
            self.ULS_check = self.ULS_check.append(pd.Series([value], index=[i]))
        self.ULS_check.name = 'Capacity'

        # Combines results of each series and brings them into 1 data frame
        df_purlin_load_parapet = pd.concat([self.snow_drift_parapet_loads, self.ULS_load_parapet, self.total_ULS_load_parapet, self.purlin_ULS_capacity, self.ULS_check], axis=1)  

        return df_purlin_load_parapet

In [5]:
class PurlinLoadExisting(SolarRoofLoading):
    
    def __init__(self, cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length):
        self.exist_ULS_load = pd.Series(dtype=float)
        self.exist_SLS_load = pd.Series(dtype=float)
        self.ULS_check = pd.Series(dtype=float)
        self.SLS_check = pd.Series(dtype=float)
        super().__init__(cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length)
        
    def purlin_load_existing_calcs(self):
        # Calculates the total existing ULS load on each purlin
        for i in range(1, self.purlin_no):
            value = self.span * self.existing_LC_BS('exist_max_1') * self.load_width[i]
            self.exist_ULS_load = self.exist_ULS_load.append(pd.Series([value], index=[i])) 
        self.exist_ULS_load.name = 'Total existing ULS load on purlins (kN)'

        # Calculates the total existing SLS load on each purlin
        for i in range(1, self.purlin_no):
            value = self.span * self.existing_LC_BS('exist_max_4') * self.load_width[i]
            self.exist_SLS_load = self.exist_SLS_load.append(pd.Series([value], index=[i])) 
        self.exist_SLS_load.name = 'Total existing SLS load on purlins (kN)'

        # Checks the calcualted loading against the ULS capacity to be less than the design load
        for i in range(1, self.purlin_no):
            if self.exist_ULS_load[i] <= self.purlin_capacities['ULS Capacity']:
                value = 'OK'
            else:
                value = 'Not OK'
            self.ULS_check = self.ULS_check.append(pd.Series([value], index=[i]))
        self.ULS_check.name = 'ULS Capacity'

        # Checks the calcualted loading against the SLS capacity to be less than the design load
        for i in range(1, self.purlin_no):
            if self.exist_SLS_load[i] <= self.purlin_capacities['SLS Capacity']:
                value = 'OK'
            else:
                value = 'Not OK'
            self.SLS_check = self.SLS_check.append(pd.Series([value], index=[i]))
        self.SLS_check.name = 'SLS Capacity'

        # Combines results of each series and brings them into 1 data frame
        df_purlin_load_existing = pd.concat([self.exist_ULS_load, self.purlin_ULS_capacity, self.ULS_check, self.exist_SLS_load, self.purlin_SLS_capacity, self.SLS_check], axis=1)  

        return df_purlin_load_existing

In [6]:
class PurlinLoadProposed(SolarRoofLoading):
    
    def __init__(self, cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length):

        # Empty dictionaries to store calculation values for the proposed loading on the purlins
        self.prop_ULS_load = pd.Series(dtype=float)
        self.prop_SLS_load = pd.Series(dtype=float)
        self.ULS_check = pd.Series(dtype=float)
        self.SLS_check = pd.Series(dtype=float)
        super().__init__(cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length)
    
    def purlin_load_proposed_calcs(self):
    
        # Calculates the total proposed ULS load on each purlin
        for i in range(1, self.purlin_no):
            value = self.span * self.proposed_LC_BS('prop_max_1') * self.load_width[i]
            self.prop_ULS_load = self.prop_ULS_load.append(pd.Series([value], index=[i])) 
        self.prop_ULS_load.name = 'Total proposed ULS load on purlins (kN)'

        # Calculates the total proposed SLS load on each purlin
        self.proposed_LC_BS('prop_max_4')
        for i in range(1,self.purlin_no):
            value = self.span * self.proposed_LC_BS('prop_max_4') * self.load_width[i]
            self.prop_SLS_load = self.prop_SLS_load.append(pd.Series([value], index=[i]))
        self.prop_SLS_load.name = 'Total proposed SLS load on purlins (kN)'  

         # Checks the calcualted loading against the ULS capacity to be less than the design load
        for i in range(1, self.purlin_no):
            if self.prop_ULS_load[i] <= self.purlin_capacities['ULS Capacity']:
                value = 'OK'
            else:
                value = 'Not OK'
            self.ULS_check = self.ULS_check.append(pd.Series([value], index=[i]))
        self.ULS_check.name = 'ULS Capacity'

        # Checks the calcualted loading against the SLS capacity to be less than the design load
        for i in range(1, self.purlin_no):
            if self.prop_SLS_load[i] <= self.purlin_capacities['ULS Capacity']:
                value = 'OK'
            else:
                value = 'Not OK'
            self.ULS_check = self.ULS_check.append(pd.Series([value], index=[i]))
        self.ULS_check.name = 'SLS Capacity'

         # Combines results of each series and brings them into 1 data frame
        df_purlin_load_proposed = pd.concat([self.purlin_ULS_capacity, self.ULS_check, self.purlin_SLS_capacity, self.SLS_check], axis=1)       

        return df_purlin_load_proposed

In [7]:
purlin_test = PurlinLoadValley(0.15,0.25,0.15,0.6,0.3,1.2,1.2,0.3,6)

Please enter the number of purlins: 5
Please enter the max purlin span: 6
Please enter purlin type: Metsec
Input the spacing between purlin 0 to purlin 1: 2


C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 1 to purlin 2: 2


C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 2 to purlin 3: 2


...

C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 4 to purlin 5: 2


C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Enter ULS Capacity of the purlins: 15
Enter SLS Capacity of the purlins: 12


C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_19108\400752581.py:51: FutureWarning: The series.append method is deprecated and 

In [17]:
purlin_test.purlin_load_valley_calcs()

C:\Users\User\AppData\Local\Temp\ipykernel_19108\1257202728.py:15: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.snow_drift_valley_loads = self.snow_drift_valley_loads.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_19108\1257202728.py:15: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.snow_drift_valley_loads = self.snow_drift_valley_loads.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_19108\1257202728.py:15: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.snow_drift_valley_loads = self.snow_drift_valley_loads.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_19108\1257202728.py:15: FutureWarning: The series.appe

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [259]:
class TestClass(SolarRoofLoading):
    
    def __init__(self, cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length):
        
        self.prop_SLS_load = pd.Series(dtype=float)
        super().__init__(cladding, services, PV_loading, maintenance, uniform_snow_loading, snow_drift_valley, snow_drift_parapet, wind_down, snow_drift_length)     
    
    def test(self):
        
        # Calculates the total proposed SLS load on each purlin
        #prop_max_4 = self.proposed_LC_BS('prop_max_4')
        for i in range(1,self.purlin_no):
            value = self.span * 4 * self.load_width[i]
            self.prop_SLS_load = self.prop_SLS_load.append(pd.Series([value], index=[i]))
        self.prop_SLS_load.name = 'Total proposed SLS load on purlins (kN)'  
        
        return self.prop_SLS_load

In [260]:
test = TestClass(0.15,0.25,0.15,0.6,0.3,1.2,1.2,0.3,6)

Please enter the number of purlins: 5
Please enter the max purlin span: 6
Please enter purlin type: metsec
Input the spacing between purlin 0 to purlin 1: 2


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 1 to purlin 2: 2


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 2 to purlin 3: 2


...

C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 4 to purlin 5: 2


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Enter ULS Capacity of the purlins: 15
Enter SLS Capacity of the purlins: 12


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will

In [258]:
test.test()

C:\Users\User\AppData\Local\Temp\ipykernel_1096\3559734923.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.prop_SLS_load = self.prop_SLS_load.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\3559734923.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.prop_SLS_load = self.prop_SLS_load.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\3559734923.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.prop_SLS_load = self.prop_SLS_load.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\3559734923.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a fut

1    20.16
2    13.44
3    13.44
4    13.44
Name: Total proposed SLS load on purlins (kN), dtype: float64

In [214]:
roof_2 = PurlinLoadExisting(0.15,0.25,0.15,0.6,0.3,1.2,1.2,0.3,6)

Please enter the number of purlins: 5
Please enter the max purlin span: 6
Please enter purlin type: mwtsec
Input the spacing between purlin 0 to purlin 1: 2


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 1 to purlin 2: 2


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 2 to purlin 3: 2


...

C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Input the spacing between purlin 4 to purlin 5: 2


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.distance_to_next_purlin = self.distance_to_next_purlin.append(pd.Series([value], index=[i]))


Enter ULS Capacity of the purlins: 15
Enter SLS Capacity of the purlins: 12


C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.purlin_ULS_capacity = self.purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_1096\400752581.py:51: FutureWarning: The series.append method is deprecated and will